This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based import *


In [2]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 0.5
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    'tau' : 0.5, 
    'edge_mask_ratio1' : 0.2,
    'edge_mask_ratio2' : 0.4,
    'feature_mask_ratio1' : 0.4, 
    'feature_mask_ratio2' : 0.2
}

generator_config = {
    'num_clusters' : 7,
}

pretext_tasks = [GRACE]

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [4]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


TypeError: 'Tensor' object is not callable